By Sakshi Verma

Importing the libraries

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Importing the data files 

In [ ]:
#File contains the imdb reviews and their corresponding 
#sentiments which can be either positive or negative

imdb_reviews=pd.read_csv("/content/imdb_reviews.csv")
test_reviews=pd.read_csv("/content/test_reviews.csv")

In [ ]:
imdb_reviews.head()

,Reviews,Sentiment
0,<START this film was just brilliant casting lo...,positive
1,<START big hair big boobs bad music and a gian...,negative
2,<START this has to be one of the worst films o...,negative
3,<START the <UNK> <UNK> at storytelling the tra...,positive
4,<START worst mistake of my life br br i picked...,negative


In [ ]:
test_reviews.head()

,Reviews,Sentiment
0,<START please give this one a miss br br <UNK>...,negative
1,<START this film requires a lot of patience be...,positive
2,<START many animation buffs consider <UNK> <UN...,positive
3,<START i generally love this type of movie how...,negative
4,<START like some other people wrote i'm a die ...,positive


Preprocessing the data:


In [ ]:
#We can not pass the string data to our model directly, so we need to transform the string data into integer format.
#We have a file that contains the mapping from words to integers so we are going to load that file.

word_index=pd.read_csv("/content/word_indexes.csv")
word_index.head()

,Words,Indexes
0,tsukino,52009
1,nunnery,52010
2,sonja,16819
3,vani,63954
4,woods,1411


In [ ]:
#Next we are going to convert the word_index dataframe into a python 
#dictionary so that we can use it for converting our reviews from string to integer format.

word_index=dict(zip(word_index.Words,word_index.Indexes))

word_index["<PAD>"]=0
word_index["<START"]=1
word_index["<UNK>"]=2
word_index["<UNUSED>"]=3

In [ ]:
#Now we define a function review_encoder that encodes the reviews into 
#integer format according to the mapping specified by word_index file.

def review_encoder(text):
  arr=[word_index[word] for word in text]
  return arr

In [ ]:
#We split the reviews from their corresponding sentiments so that we can 
#preprocess the reviews and sentiments separately and then later pass it to our model.

train_data,train_labels=imdb_reviews['Reviews'],imdb_reviews['Sentiment']
test_data, test_labels=test_reviews['Reviews'],test_reviews['Sentiment']

In [ ]:
#Before transforming the reviews as integers we need to tokenize or split the review on the basis of whitespaces

train_data=train_data.apply(lambda review:review.split())
test_data=test_data.apply(lambda review:review.split())

In [ ]:
#Since we have tokenized the reviews now we can apply the review_encoder 
#function to each review and transform the reviews into integer format.

train_data=train_data.apply(review_encoder)
test_data=test_data.apply(review_encoder)

In [ ]:
train_data.head()

0    [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, ...
1    [1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463,...
2    [1, 14, 47, 8, 30, 31, 7, 4, 249, 108, 7, 4, 5...
3    [1, 4, 2, 2, 33, 2804, 4, 2040, 432, 111, 153,...
4    [1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 1...
Name: Reviews, dtype: object

In [ ]:
#We also need to encode the sentiments and we are labeling the positive sentiment as 1 and negative sentiment as 0.

def encode_sentiments(x):
  if x=='positive':
    return 1
  else:
    return 0

train_labels=train_labels.apply(encode_sentiments)
test_labels=test_labels.apply(encode_sentiments)

In [ ]:
#Before giving the review as an input to the model we need to perform following preprocessing steps:

#The length of each review should be made equal for the model to be working correctly.
#We have chosen the length of each review to be 500.

train_data=keras.preprocessing.sequence.pad_sequences(train_data,value=word_index["<PAD>"],padding='post',maxlen=500)
test_data=keras.preprocessing.sequence.pad_sequences(test_data,value=word_index["<PAD>"],padding='post',maxlen=500)


Building the model

In [ ]:
#one word embedding layer which creates word embeddings of length 16 from integer encoded review.
#second layer is global average pooling layer which is used to prevent overfitting by reducing the number of parameters.
#then a dense layer which has 16 hidden units and uses relu as activation function
#the final layer is the output layer which uses sigmoid as activation function

model=keras.Sequential([keras.layers.Embedding(10000,16,input_length=500),
                        keras.layers.GlobalAveragePooling1D(),
                        keras.layers.Dense(16,activation='relu'),
                        keras.layers.Dense(1,activation='sigmoid')])

Compiling the model

In [ ]:
#Adam is used as optimization function for our model.
#Binary cross entropy loss function is used as loss function for the model.
#Accuracy is used as the metric for evaluating the model.

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
#training the model
history=model.fit(train_data,train_labels,epochs=30,batch_size=512,validation_data=(test_data,test_labels))

Epoch 1/30
49/49 [==============================] - 5s 79ms/step - loss: 0.6919 - accuracy: 0.5383 - val_loss: 0.6898 - val_accuracy: 0.5365
Epoch 2/30
49/49 [==============================] - 3s 60ms/step - loss: 0.6849 - accuracy: 0.6401 - val_loss: 0.6795 - val_accuracy: 0.6808
Epoch 3/30
49/49 [==============================] - 2s 34ms/step - loss: 0.6665 - accuracy: 0.7150 - val_loss: 0.6547 - val_accuracy: 0.7512
Epoch 4/30
49/49 [==============================] - 2s 32ms/step - loss: 0.6311 - accuracy: 0.7711 - val_loss: 0.6148 - val_accuracy: 0.7810
Epoch 5/30
49/49 [==============================] - 2s 33ms/step - loss: 0.5798 - accuracy: 0.8031 - val_loss: 0.5617 - val_accuracy: 0.8070
Epoch 6/30
49/49 [==============================] - 2s 33ms/step - loss: 0.5189 - accuracy: 0.8329 - val_loss: 0.5067 - val_accuracy: 0.8226
Epoch 7/30
49/49 [==============================] - 2s 34ms/step - loss: 0.4604 - accuracy: 0.8512 - val_loss: 0.4575 - val_accuracy: 0.8399
Epoch 8/30
49

In [ ]:
#Evaluating the model

loss,accuracy=model.evaluate(test_data,test_labels)

782/782 [==============================] - 2s 2ms/step - loss: 0.2925 - accuracy: 0.8866


Testing our model

In [ ]:
index=np.random.randint(1,1000)
user_review=test_reviews.loc[index]
print(user_review)

Reviews      <START another great tom <UNK> performance <UN...
Sentiment                                             positive
Name: 371, dtype: object


In [ ]:
user_review=test_data[index]
user_review=np.array([user_review])
if (model.predict(user_review)>0.5).astype("int32"):
  print("positive sentiment")
else:
  print("negative sentiment")


positive sentiment
